# Projeto 01 - Transcrição e compreensão de vídeos

Neste projeto, vamos aprender a realizar transcrição e compreensão de vídeos. Ao final, você será capaz de criar sua própria aplicação que faz a sumarização automática de vídeos, permitindo que você entenda do que se trata e o que foi falado nele sem precisar assistí-lo.

Objetivos deste projeto:

* Compreender o conteúdo de um vídeo do Youtube sem precisar assisti-lo.
* Pesquisar informações úteis no vídeo sem perder nenhum detalhe importante.
* Interagir com o conteúdo do vídeo por meio de uma interface de chat (ou seja, como "conversar com o vídeo").

## Instalação e Configuração

In [ ]:
!pip install -q langchain_community langchain-huggingface langchain_ollama langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.

### Instalação de bibliotecas para baixar transcrição

> **YouTube Transcript API**

Esta é uma API python que permite que você obtenha a transcrição/legendas para um determinado vídeo do YouTube. Ela também funciona para legendas geradas automaticamente e possui suporta a uma função que faz automaticamente a tradução de legendas

In [ ]:
!pip install youtube-transcript-api

> **pytube**

Também é uma biblioteca que auxilia com o download de vídeos no youtube. Aqui ela não é necessária para baixar as transcrições dos vídeos, conseguimos ter acesso sem ela, mas iremos instalar também pois com ela podemos recuperar também demais informações do vídeo, como título, data de publicação, descrição, etc.

In [ ]:
!pip install pytube

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00


## Importações

In [ ]:
import os
import io
import getpass
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.llms import HuggingFaceHub
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

## Carregando a transcrição

Para fazer o carregamento das transcrições usaremos o método YoutubeLoader(), que faz parte do dos document_loaders do LangChain. Veremos logo em seguida também como extrair os metadados do vídeo usando essa função

Através desses método conseguimos puxar as transcrições que já estão associadas ao vídeo e armazenadas no banco de dados do Youtube, o que irá nos economizar bastante processamento.


O primeiro parâmetro é a URL do vídeo que queremos realizar a transcrição

Vamos pegar esse vídeo como primeiro exemplo https://www.youtube.com/watch?v=II28i__Tf3M

### Defininido idiomas

O segundo parâmetro é o language. A função espera uma lista, nesse caso, uma lista de códigos de idioma em prioridade decrescente (por padrão).

Além de inglês ("en"), recomendamos deixar antes "pt" e "pt-BR" (ou "pt-PT") pois em alguns vídeos não possui "pt". Embora a grande maioria dos vídeos que testamos possua legenda com código "pt", mesmo para vídeos com a legenda em português brasileiro. Ou seja, deixamos assim pois em alguns vídeos do português brasileiro por exemplo o código é "pt", já para outros está como "pt-BR".


In [ ]:
video_loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=II28i__Tf3M",
                                              language = ["pt", "pt-BR", "en"],)

Usaremos o .load() para fazer a leitura e ao mesmo tempo podemos passar as informações do vídeo para uma variável

In [ ]:
infos = video_loader.load()
infos

[Document(metadata={'source': 'II28i__Tf3M'}, page_content='Olá sejam bem-vindos a sala que você vai aprender alguns fundamentos biológicos básicos sobre as redes neurais Primeiramente você verá sobre as redes neurais humana na sequência Vamos trabalhar com as redes neurais artificiais nós temos essa imagem representa os neurônios do cérebro existem bilhões de neurônios que estão conectados base nesta imagem nós vamos discutir três principais o primeiro ponto é que existem muitos neurônios o segundo ponto é que eles estão conectados entre si e o terceiro ponto é que estes neurônio por meio dessas conexões trocam informações entre si conexão entre os neurônios dos é e é responsável pelas nossas habilidades por exemplo ver falar dar e assim por diante vamos supor que você fala o idioma português e se indica que essas conexões entre os neurônios estão criadas de forma a permitir você falar esse idioma vamos supor que você começa a aprender a falar inglês com isso novas conexões Entre esse

O valor de "page_content" corresponde à transcrição em si

para acessá-la devemos colocar `[0]` pois `infos` é uma lista, que nesse caso só tem o primeiro valor. O código então fica assim

In [ ]:
transcricao = infos[0].page_content
transcricao

'Olá sejam bem-vindos a sala que você vai aprender alguns fundamentos biológicos básicos sobre as redes neurais Primeiramente você verá sobre as redes neurais humana na sequência Vamos trabalhar com as redes neurais artificiais nós temos essa imagem representa os neurônios do cérebro existem bilhões de neurônios que estão conectados base nesta imagem nós vamos discutir três principais o primeiro ponto é que existem muitos neurônios o segundo ponto é que eles estão conectados entre si e o terceiro ponto é que estes neurônio por meio dessas conexões trocam informações entre si conexão entre os neurônios dos é e é responsável pelas nossas habilidades por exemplo ver falar dar e assim por diante vamos supor que você fala o idioma português e se indica que essas conexões entre os neurônios estão criadas de forma a permitir você falar esse idioma vamos supor que você começa a aprender a falar inglês com isso novas conexões Entre esses neurônios são geradas para permitir que você consiga fala

Esse primeiro exemplo é de uma legenda que foi gerada automaticamente pelo sistema de reconhecimento de fala do youtube, que no geral tende a ser bom mas pode gerar erros, então não é perfeito. Mas ainda assim, dependendo da LLM ela vai entender que se trata de um erro com base no contexto

Para legendas automáticas verificamos que não houve perda considerável na compreensão, mas obviamento é esperado que uma legenda feita manualmente possua maiores chances de resultados melhores

### Obter informações do vídeo

Note que carregamos a legenda/transcrição mas nenhuma outra informação sobre o vídeo, o que pode ser útil depedendo do nosso objetivo.

Podemos passar como parâmetro add_video_info=True (que por padrão é =False) e isso fará com que sejam retornados os metadados do vídeo, como: título, descrição, autor, visualizações, e capa)

Para usar esse parâmetro você precisa ter instalado antes a biblioteca pytube

In [ ]:
video_loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=II28i__Tf3M",
                                               add_video_info = True,
                                               language = ["pt", "pt-BR", "en"],)

In [ ]:
infos = video_loader.load()

In [ ]:
infos

[Document(metadata={'source': 'II28i__Tf3M', 'title': 'Introdução aos fundamentos biológicos das redes neurais artificiais', 'description': 'Unknown', 'view_count': 5473, 'thumbnail_url': 'https://i.ytimg.com/vi/II28i__Tf3M/hq720.jpg', 'publish_date': '2022-03-29 00:00:00', 'length': 287, 'author': 'IA Expert Academy'}, page_content='Olá sejam bem-vindos a sala que você vai aprender alguns fundamentos biológicos básicos sobre as redes neurais Primeiramente você verá sobre as redes neurais humana na sequência Vamos trabalhar com as redes neurais artificiais nós temos essa imagem representa os neurônios do cérebro existem bilhões de neurônios que estão conectados base nesta imagem nós vamos discutir três principais o primeiro ponto é que existem muitos neurônios o segundo ponto é que eles estão conectados entre si e o terceiro ponto é que estes neurônio por meio dessas conexões trocam informações entre si conexão entre os neurônios dos é e é responsável pelas nossas habilidades por exemp

Podemos organizar desse modo

In [ ]:
infos_video = f"""Informações do vídeo:

Título: {infos[0].metadata['title']}
Autor: {infos[0].metadata['author']}
Data: {infos[0].metadata['publish_date'][:10]}
URL: https://www.youtube.com/watch?v={infos[0].metadata['source']}

Transcrição: {transcricao}
"""
print(infos_video)

Informações do vídeo:

Título: Introdução aos fundamentos biológicos das redes neurais artificiais
Autor: IA Expert Academy
Data: 2022-03-29
URL: https://www.youtube.com/watch?v=II28i__Tf3M

Transcrição: Olá sejam bem-vindos a sala que você vai aprender alguns fundamentos biológicos básicos sobre as redes neurais Primeiramente você verá sobre as redes neurais humana na sequência Vamos trabalhar com as redes neurais artificiais nós temos essa imagem representa os neurônios do cérebro existem bilhões de neurônios que estão conectados base nesta imagem nós vamos discutir três principais o primeiro ponto é que existem muitos neurônios o segundo ponto é que eles estão conectados entre si e o terceiro ponto é que estes neurônio por meio dessas conexões trocam informações entre si conexão entre os neurônios dos é e é responsável pelas nossas habilidades por exemplo ver falar dar e assim por diante vamos supor que você fala o idioma português e se indica que essas conexões entre os neurônios e

## Salvando transcrição em um arquivo

Esse código abre um arquivo chamado "transcricao.txt" em modo de escrita ("w") com codificação UTF-8;  dentro do bloco `with` ele grava dados no arquivo.

Para cada item na variável `infos`, ele escreve o conteúdo da variável `infos_video` no arquivo. O uso do bloco with garante que o arquivo seja fechado corretamente após a gravação, mesmo que ocorra algum erro durante a execução. E aqui não precisa do `f.close()` pois o bloco with fecha o arquivo automaticamente ao finalizar

In [ ]:
with io.open("transcricao.txt", "w", encoding="utf-8") as f:
  for doc in infos:
    f.write(infos_video)

## Carregamento do modelo

Vamos reaproveitar as funções de carregamento que usamos nos projeos anteriores, basta copiar e colar

In [ ]:
def model_hf_hub(model = "meta-llama/Meta-Llama-3-8B-Instruct", temperature = 0.1):
  llm = HuggingFaceHub(repo_id = model,
                       model_kwargs={
                           "temperature": temperature,
                           "return_full_text": False,
                           "max_new_tokens": 1024,
                       })
  return llm

In [ ]:
def model_openai(model = "gpt-4o-mini", temperature = 0.1):
  llm = ChatOpenAI(model = model, temperature = temperature)
  return llm

In [ ]:
def model_ollama(model = "phi3", temperature = 0.1):
  llm = ChatOllama(model = model, temperature = temperature)
  return llm

E aqui no Colab precisar setar as variáveis de ambiente. Pode usar o .env também, especialmente se estiver executando localmente

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass()

··········


In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
model_class = "hf_hub" # @param ["hf_hub", "openai", "ollama"]

if model_class == "hf_hub":
  llm = model_hf_hub()
elif model_class == "openai":
  llm = model_openai
elif model_class == "ollama":
  llm = model_ollama


<ipython-input-13-7c8108b6e615>:2: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  llm = HuggingFaceHub(repo_id = model,


In [ ]:
model_class, llm

('hf_hub',
 HuggingFaceHub(client=<InferenceClient(model='meta-llama/Meta-Llama-3-8B-Instruct', timeout=None)>, repo_id='meta-llama/Meta-Llama-3-8B-Instruct', task='text-generation', model_kwargs={'temperature': 0.1, 'return_full_text': False, 'max_new_tokens': 1024}))

## Criação do prompt template

Vamos manter a base do prompt simples, basicamente instruindo que deve responder com base na transcrição fornecida. Você pode modificá-lo à vontade depois, para deixar mais adequado ao seu objetivo ou simplesmente para tentar alcançar melhores resultados

* Aqui vamos passar o transcrição completa. Estaremos lidando com modelos que possuem uma janela grande de contexto - por exemplo o llama 3 possui algo em torno de 8k, já o chatGPT 4o por exemplo possui ainda mais. Deve ser uma capacidade suficiente de leitura de tokens de entrada para lidar com a maioria das transcrições dos vídeos, e será para todos os testados aqui.
* Como a ideia desse projeto é criar uma ferramenta que faz o resumo / sumarização então adicionar a transcrição inteira como contexto é até uma opção mais interessante, já que para RAG é recuperado geralmente uma quantidade limite de pedaços de documento. Portanto, se fosse usado RAG teria que configurar bem os parâmetros, provavelmente escolher um valor maior de k por exemplo para recuperar mais documentos (no entanto, lembre-se que elevar muito esse valor aumenta o custo computacional da aplicação)
* Mas caso o vídeo seja realmente grande então pode ser interessante dividir em partes. Para isso sugerimos usar o código do projeto 3, pode copiar as funções prontas que fazem as etapas de indexação e recuperação (indexing & retrieval)

Além da transcrição, o prompt template irá aceitar a variável consulta, que nada mais é do que a entrada para a LLM, que pode ser uma pergunta ou instrução

E o `if model_class.startswith("hf"):` apenas copiamos do projeto anterior, lembrando que isso é para melhorar os resultados com a implementação via Hugging Face Hub, que até o momento funciona melhor se especificarmos manualmente os tokens de início e fim. Aqui o template é do llama 3.x, mas for usar outro modelo open source que exija template diferente então lembre de mudar.


In [ ]:
system_prompt = "Você é um assistente virtual prestativo e deve responder a uma consulta com base na transcrição de um vídeo, que será fornecida abaixo."

inputs = "Consulta: {consulta} \n Transcrição: {transcricao}"

if model_class.startswith("hf"):
  user_prompt = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>".format(inputs)
else:
  user_prompt = "{}".format(inputs)

prompt_template = ChatPromptTemplate.from_messages([("system", system_prompt), ("user", user_prompt)])

In [ ]:
prompt_template

ChatPromptTemplate(input_variables=['consulta', 'transcricao'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Você é um assistente virtual prestativo e deve responder a uma consulta com base na transcrição de um vídeo, que será fornecida abaixo.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['consulta', 'transcricao'], template='<|begin_of_text|><|start_header_id|>user<|end_header_id|>\nConsulta: {consulta} \n Transcrição: {transcricao}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'))])

## Criação da chain

Nossa chain ficará assim

In [ ]:
chain = prompt_template | llm | StrOutputParser()

## Geração da resposta

Por fim, vamos gerar o resultado, fornecendo como parâmetro a transcrição e a consulta que queremos (podendo ser pergunta, instrução, etc.)

In [ ]:
res = chain.invoke({"transcricao": transcricao, "consulta": "resuma"})
print(res)



Aqui está uma resumo da transcrição:

A aula começa apresentando a estrutura do cérebro humano, com bilhões de neurônios conectados entre si. Os neurônios são responsáveis pelas habilidades humanas, como falar, ver e ouvir. A troca de informações entre os neurônios é feita por meio de conexões sinápticas, que permitem que os neurônios aprendam novas habilidades.

O neurônio é composto por três principais componentes: dendritos, corpo celular e axônio. Os dendritos recebem informações, o corpo celular processa a informação e o axônio transmite a informação processada para outros neurônios.

A rede neural é formada pela conexão entre vários neurônios, permitindo a troca de informações entre eles. A sinapse é o processo de troca de informações entre os neurônios, que ocorre quando o potencial elétrico do corpo celular do neurônio é alterado.

A aula conclui apresentando os principais conceitos biológicos sobre as redes neurais, preparando o aluno para entender como funcionam os neurônio

Podemos melhorar esse prompt (consulta), deixando algo como `"sumarize de forma clara de entender`

In [ ]:
res = chain.invoke({"transcricao": transcricao, "consulta": "sumarize de forma clara de entender"})
print(res)



Aqui está uma síntese clara e fácil de entender da transcrição:

As redes neurais são compostas por bilhões de neurônios que se conectam entre si, permitindo a troca de informações. Existem três principais características: muitos neurônios, conexões entre eles e troca de informações. Essas conexões são responsáveis pelas nossas habilidades, como falar, ver e ouvir.

Um neurônio é composto por dendritos (que recebem informações), corpo celular (onde a informação é processada) e axónio (que transmite a informação processada para outro neurônio). A troca de informações entre neurônios é chamada de sinapse, que ocorre quando o potencial elétrico do corpo celular do neurônio é alterado.

A rede neural é formada pela conexão entre vários neurônios, permitindo a troca de informações entre eles. A informação chega aos dendritos, é processada no corpo celular e é enviada para outros neurônios por meio dos terminais do axónio. Essa conexão é fundamental para o funcionamento das redes neurais, 

In [ ]:
res = chain.invoke({"transcricao": transcricao, "consulta": "explique em 1 frase sobre o que fala esse vídeo"})
print(res)



O vídeo explica os fundamentos biológicos básicos sobre as redes neurais, começando com a estrutura do cérebro humano e como os neurônios se conectam e trocam informações entre si.


In [ ]:
res = chain.invoke({"transcricao": transcricao, "consulta": "liste os temas desse video"})
print(res)



Aqui estão os temas do vídeo:

1. Redes neurais humanas
2. Componentes do neurônio (dendritos, corpo celular, axónio, terminais do axónio)
3. Sinapse (troca de informações entre neurônios)
4. Processamento de informações no cérebro humano
5. Redes neurais artificiais
6. Aprendizado e formação de conexões entre neurônios
7. Simulação de redes neurais com vários neurônios conectados.


## Tradução da transcrição

Para os modelos mais modernos não é necessário traduzir antes, pode carregar a transcrição no idioma desejado e passar para a LLM mesmo que no idioma diferente daquele que você escreveu as instruções no prompt template, isso porque o modelo deve ser capaz de entender.

Mas também é possível traduzir a transcrição usando essa mesma ferramenta.
Isso pode ser muito útil caso o modelo que esteja trabalhando não funcione bem para múltiplos idiomas.

Para implementar isso, basta definirmos para o parâmetro translation o código do idioma para o qual desejamos traduzir.
Por exemplo para o francês ficaria assim

In [ ]:
url_video = "https://www.youtube.com/watch?v=II28i__Tf3M"

video_loader = YoutubeLoader.from_youtube_url(
    url_video,
    add_video_info=True,
    language=["pt", "en"],
    translation="fr",
)

infos = video_loader.load()
transcricao = infos[0].page_content
transcricao

"Bonjour, bienvenue dans la chambre où vous allez apprendre quelques fondamentaux biologiques bases sur les réseaux de neurones Vous verrez d’abord sur les réseaux séquence neuronale humaine Allons-y travailler avec des réseaux de neurones artificiel nous avons cette image représente les neurones du cerveau il y a des milliards de neurones qui sont en ligne, sur la base de cette image, nous allons discuter de trois principaux, le premier le fait est qu'il y a beaucoup de neurones le deuxième point est qu'ils sont connectés les uns aux autres et le troisième point est que ces neurones à travers ces relations échanger des informations entre eux connexion entre les neurones de l'est et de l'est responsable de nos compétences par exemple voir parler donner et ainsi de suite Supposons que vous parlez langue portugaise et il est indiqué que ces les connexions entre les neurones sont créé pour vous permettre de parler cette langue, supposons que vous commenciez apprenez à parler anglais avec 

## Junção da pipeline em funções

Para deixar mais prático e evitar repetições do código vamos reunir toda a nossa lógica em funções, assim não vai ser mais necessário ficar copiando e colando o código toda vez que for testar em outro vídeo

In [ ]:
def llm_chain(model_class):
  system_prompt = "Você é um assistente virtual prestativo e deve responder a uma consulta com base na transcrição de um vídeo, que será fornecida abaixo."

  inputs = "Consulta: {consulta} \n Transcrição: {transcricao}"

  if model_class.startswith("hf"):
      user_prompt = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>".format(inputs)
  else:
      user_prompt = "{}".format(inputs)

  prompt_template = ChatPromptTemplate.from_messages([
      ("system", system_prompt),
      ("user", user_prompt)
  ])

  ### Carregamento da LLM
  if model_class == "hf_hub":
      llm = model_hf_hub()
  elif model_class == "openai":
      llm = model_openai()
  elif model_class == "ollama":
      llm = model_ollama()

  chain = prompt_template | llm | StrOutputParser()

  return chain

In [ ]:
def get_video_info(url_video, language="pt", translation=None):

  video_loader = YoutubeLoader.from_youtube_url(
      url_video,
      add_video_info=True,
      language=language,
      translation=translation,
  )

  infos = video_loader.load()[0]
  metadata = infos.metadata
  transcript = infos.page_content

  return transcript, metadata

Vamos testar abaixo

In [ ]:
transcript, metadata = get_video_info("https://www.youtube.com/watch?v=II28i__Tf3M")

In [ ]:
metadata, transcript

({'source': 'II28i__Tf3M',
  'title': 'Introdução aos fundamentos biológicos das redes neurais artificiais',
  'description': 'Unknown',
  'view_count': 5473,
  'thumbnail_url': 'https://i.ytimg.com/vi/II28i__Tf3M/hq720.jpg',
  'publish_date': '2022-03-29 00:00:00',
  'length': 287,
  'author': 'IA Expert Academy'},
 'Olá sejam bem-vindos a sala que você vai aprender alguns fundamentos biológicos básicos sobre as redes neurais Primeiramente você verá sobre as redes neurais humana na sequência Vamos trabalhar com as redes neurais artificiais nós temos essa imagem representa os neurônios do cérebro existem bilhões de neurônios que estão conectados base nesta imagem nós vamos discutir três principais o primeiro ponto é que existem muitos neurônios o segundo ponto é que eles estão conectados entre si e o terceiro ponto é que estes neurônio por meio dessas conexões trocam informações entre si conexão entre os neurônios dos é e é responsável pelas nossas habilidades por exemplo ver falar dar

Aqui aproveitamos para adicionar um Tratamento de erro com Try Catch, pois caso não haja uma transcrição para esse vídeo será retornado um erro (possivelmente será esse: `IndexError: list index out of range`), com isso não será possível fazer os processamentos seguintes. Por isso programos aqui para o programa parar interromper a execuão se esse for caso

In [ ]:
def interpret_video(url, query="resuma", model_class="hf_hub", language="pt", translation=None):

  try:
    transcript, metadata = get_video_info(url, language, translation)

    chain = llm_chain(model_class)

    res = chain.invoke({"transcricao": transcript, "consulta": query})
    print(res)

  except Exception as e:
    print("Erro ao carregar transcrição")
    print(e)

## Geração final

Podemos definir uma interface mais apresentável no Colab através dos comandos para deixar as variáveis em formato de valores de formulário.

Ideias do que adicionar à query:

* `sumarize de forma clara de entender`
* `liste os temas desse vídeo`
* `explique em 1 frase sobre o que fala esse vídeo`

In [ ]:
url_video = "https://www.youtube.com/watch?v=II28i__Tf3M" # @param {type:"string"}
query_user = "sumarize de forma clara de entender" # @param {type:"string"}
model_class = "openai" # @param ["hf_hub", "openai", "ollama"]
language = ["pt", "pt-BR", "en"] # @param {type:"string"}

In [ ]:
interpret_video(url_video, query_user, model_class, language)

O vídeo apresenta uma introdução aos fundamentos biológicos das redes neurais, começando com os neurônios humanos. Ele destaca três pontos principais: 

1. Existem bilhões de neurônios no cérebro.
2. Esses neurônios estão interconectados.
3. Através dessas conexões, os neurônios trocam informações, o que é essencial para habilidades como falar e aprender novos idiomas.

O vídeo explica a estrutura de um neurônio, que inclui dendritos (que recebem informações), o corpo celular (onde a informação é processada) e o axônio (que transmite a informação para outros neurônios). A troca de informações entre neurônios ocorre nas sinapses, onde processos elétricos são ativados.

Por fim, o vídeo promete uma próxima aula sobre o funcionamento dos neurônios artificiais.


In [ ]:
url_video = "https://www.youtube.com/watch?v=rEE8ERGKsqo" # @param {type:"string"}
query_user = "sumarize de forma clara de entender" # @param {type:"string"}
model_class = "hf_hub" # @param ["hf_hub", "openai", "ollama"]
language = ["pt", "pt-BR", "en"] # @param {type:"string"}

interpret_video(url_video, query_user, model_class, language)



Aqui está uma síntese clara e fácil de entender do vídeo:

A astronomia é o estudo do universo, incluindo objetos que podemos ver com nossos próprios olhos, como o sol, a lua, os planetas e as estrelas, bem como objetos que só podemos ver com telescópios especiais. A ciência é muito antiga e tem passado por muitas transformações ao longo do tempo.

O universo é muito grande e está em constante movimento. A luz é a unidade de medida usada para medir as distâncias entre os astros. O sistema solar é o grupo de objetos que inclui a Terra e o sol, e é composto por oito planetas, cinco planetas anões, mais de 200 luas, milhares de cometas e outros objetos espaciais.

As estrelas são enormes bolas de gás queimando e variam em tamanho, cor e brilho. Elas podem ser vermelhas, laranjas, amarelas, brancas ou azuis, dependendo da temperatura de sua superfície. As estrelas também passam por muitos estágios em suas vidas, incluindo a formação em nuvens de poeira e gás, a brilhanteza por até 10 bil

## Explorando mais

Vamos deixar nossa aplicação mais interessante. Podemos fazer mais de uma consulta/query de uma vez por vídeo.

* Para cada vídeo informado, podemos definir para exibir informações como o titulo dele;
* e logo abaixo um resumo em um paragrafo;
* depois uma lista de temas abordados, etc.

No Colab temos um modo interessante de fazer isso, que é através do Markdown.


### Exemplo com Markdown

Markdown é um formato de linguagem de marcação muito adotada pela comunidade por ser simples e leve, permitindo criar texto formatado com uso de símbolos especiais, como asteriscos e colchetes, em vez de tags de HTML. No Google Colab, o uso de markdown pode tornar a visualização do texto mais interessante e fácil de ler, facilitando a compreensão e a apresentação de informações. Por exemplo, usando markdown, você pode deixar um texto em *itálico* ao deixar dentro de asteriscos, ou **negrito** se deixar ele dentro de asteriscos duplos. Também podemos adicionar títulos com diferentes níveis, por exemplo para nível 1,2,3 basta colocar antes da frase `#` `##` ou `###` respectivamente

> Mais sobre a sintaxe aqui: https://www.markdownguide.org/basic-syntax/

In [ ]:
texto = """
### Título
descrição em **destaque** *aqui...*
"""

In [ ]:
from IPython.display import Markdown

display(Markdown(texto))


### Título
descrição em **destaque** *aqui...*


E abaixo em forma de lista por exemplo

Com isso, se pedirmos por exemplo pra LLM retornar uma lista então ficará mais apresentável também, já que ela irá retornar nesse formato com * no inicio de cada item, já que é um padrão comum

Nos projetos em que usamos os Streamlit talvez tenha notada que já ficou automaticamente dessem modo, pois a interface já interpreta corretamente markdown

In [ ]:
lista = """
* item
* item
* item
"""

In [ ]:
display(Markdown(lista))


* item
* item
* item


### Finalização do código

Copiamos do código que fizemos antes, só mudando de infos[0].metadata para metadata

e ao invés do print() normal, vamos colocar no lugar display(Markdown()) para deixar mais apresentável

Como resposta, teremos:
* Informações do vídeo - que são os metadados: título, autor, data, URL.

*  Sobre o que fala o vídeo - resumo de 1 frase, para contextualizarmos bem rapidamente

* Temas - listagem dos principais temas desse vídeo

* Resposta para a consulta - que é a resposta para a consulta personalizada que fizemos, que pode ser uma pergunta ou instrução

Aqui você pode customizar à vontade depois, deixar com as consultas que achar mais conveniente para o objetivo de sua aplicação

Você poderia também separar em duas funções: uma para exibir junto as consultas fixas (informações do vídeo, resumo e temas) e outra para exibir a consulta personalizada, podendo deixar em formato de chat, igual feito no projeto 1 e 2.

mas aqui estamos deixando mais simples e rápido, portanto ficou desse modo

In [ ]:
def interpret_video(url, query="liste os temas desse vídeo", model_class="hf_hub", language="pt", translation=None):

  try:
    transcript, metadata = get_video_info(url, language, translation)

    infos_video = f"""## Informações do vídeo

    Título: {metadata['title']}
    Autor: {metadata['author']}
    Data: {metadata['publish_date'][:10]}
    URL: https://www.youtube.com/watch?v={metadata['source']}

    """

    display(Markdown(infos_video))

    chain = llm_chain(model_class)

    t = "\n## Sobre o que fala o vídeo \n"
    res = chain.invoke({"transcricao": transcript, "consulta": "explique em 1 frase sobre o que fala esse vídeo. responda direto com a frase"})
    display(Markdown(t + res))

    t = "\n## Temas \n"
    res = chain.invoke({"transcricao": transcript, "consulta": "lista os principais temas desse vídeo"})
    display(Markdown(t + res))

    t = "\n## Resposta para a consulta \n"
    res = chain.invoke({"transcricao": transcript, "consulta": query})
    display(Markdown(t + res))

  except Exception as e:
    print("Erro ao carregar transcrição")
    print(e)

In [ ]:
url_video = "https://www.youtube.com/watch?v=rEE8ERGKsqo" # @param {type:"string"}
query_user = "sumarize de forma clara de entender" # @param {type:"string"}
model_class = "hf_hub" # @param ["hf_hub", "openai", "ollama"]
language = ["pt", "pt-BR", "en"] # @param {type:"string"}

interpret_video(url_video, query_user, model_class, language)

## Informações do vídeo

    Título: TUDO QUE VOCÊ PRECISA SABER PRA ENTENDER O BÁSICO SOBRE ASTRONOMIA
    Autor: Universo Interessado
    Data: 2023-03-26
    URL: https://www.youtube.com/watch?v=rEE8ERGKsqo

    


## Sobre o que fala o vídeo 


Aqui está a frase que resume o vídeo: "O vídeo explica o básico da astronomia, incluindo o estudo do universo, estrelas, planetas, galáxias e a expansão do universo."


## Temas 


Aqui estão os principais temas do vídeo:

1. Introdução à astronomia: definição, história e importância da ciência dos astros.
2. Conceitos básicos da astronomia: universo, sistema solar, estrelas, galáxias, matéria escura e energia escura.
3. O universo: sua idade, expansão e evolução.
4. O sistema solar: planetas, estrelas, luas e asteroides.
5. Estrelas: suas características, ciclos de vida e tipos (vermelhas, amarelas, brancas, etc.).
6. Galáxias: suas características, tipos (espiral, elíptica, celular) e importância.
7. Constelações: sua definição, história e importância.
8. Observatórios e telescópios: sua função, tipos e importância para a astronomia.
9. Astrônomos: sua definição, papéis e importância para a astronomia.
10. Cidadãos astrônomos: sua participação e contribuição para a astronomia.


## Resposta para a consulta 


Aqui está uma síntese clara e fácil de entender do vídeo:

A astronomia é o estudo do universo, incluindo objetos que podemos ver com nossos próprios olhos, como o sol, a lua, os planetas e as estrelas, bem como objetos que só podemos ver com telescópios especiais. A ciência é muito antiga e tem passado por muitas transformações ao longo do tempo.

O universo é muito grande e está em constante movimento. A luz é a unidade de medida usada para medir as distâncias entre os astros. O sistema solar é o grupo de objetos que inclui a Terra e o sol, e é composto por oito planetas, cinco planetas anões, mais de 200 luas, milhares de cometas e outros objetos espaciais.

As estrelas são enormes bolas de gás queimando e variam em tamanho, cor e brilho. Elas podem ser vermelhas, laranjas, amarelas, brancas ou azuis, dependendo da temperatura de sua superfície. As estrelas também passam por muitos estágios em suas vidas, incluindo a formação em nuvens de poeira e gás, a brilhanteza por até 10 bilhões de anos e a explosão em uma supernova.

As galáxias são coleções de bilhões de estrelas, gás e poeira mantidas juntas pela gravidade. A Via Láctea é a galáxia em que a Terra está localizada e é composta por cerca de 100 bilhões de estrelas. A matéria escura e a energia escura são dois conceitos que os cientistas ainda não entendem muito bem, mas sabem que elas compõem 95% do universo.

Os observatórios e telescópios são ferramentas essenciais para os astrônomos, permitindo que eles vejam o céu com mais detalhes do que o olho humano permite. Os astrônomos podem ser observacionais, teóricos ou cidadãos, e todos jogam um papel importante na compreensão do universo.

Espero que isso tenha ajudado!

In [ ]:
url_video = "https://www.youtube.com/watch?v=n9u-TITxwoM" # @param {type:"string"}
query_user = "sumarize de forma clara de entender" # @param {type:"string"}
model_class = "hf_hub" # @param ["hf_hub", "openai", "ollama"]
language = ["pt", "pt-BR", "en"] # @param {type:"string"}

interpret_video(url_video, query_user, model_class, language)

## Informações do vídeo

    Título: The six degrees | Kevin Bacon | TEDxMidwest
    Autor: TEDx Talks
    Data: 2012-06-27
    URL: https://www.youtube.com/watch?v=n9u-TITxwoM

    


## Sobre o que fala o vídeo 


Aqui está a frase que resume o vídeo: "O ator Kevin Bacon fala sobre a origem do jogo 'The Six Degrees of Kevin Bacon' e como ele se tornou um ativista para a filantropia, criando o site sixdegrees.org para conectar pessoas e causas."


## Temas 


Os principais temas desse vídeo são:

1. A origem do jogo "Six Degrees of Kevin Bacon" e como ele se tornou um fenômeno popular.
2. A filantropia e a importância da conectividade e da divulgação de causas sociais.
3. A criação do site sixdegrees.org e a ideia de celebridades se tornarem "rosto" de causas sociais.
4. A importância da participação das pessoas comuns em causas sociais e a possibilidade de se tornarem "celebridades" de suas próprias causas.
5. A evolução do site e a busca por novas ideias e formas de usar a internet como uma força para o bem.


## Resposta para a consulta 


O ator Kevin Bacon compartilha sua experiência com o jogo "The Six Degrees of Kevin Bacon" e como isso o levou a criar a Fundação Footloose, que se tornou a Network for Good. Ele começa contando como descobriu o jogo e como isso o fez se sentir inseguro e preocupado com a possibilidade de ser ridicularizado. No entanto, após uma conversa com os criadores do jogo, ele se sentiu mais à vontade e começou a trabalhar com eles para desenvolver o site sixdegrees.org.

Ele descreve como o site foi criado para conectar celebridades com causas sociais e permitir que as pessoas doassem dinheiro para essas causas. No entanto, o site não estava recebendo o dinheiro que esperavam e Kevin Bacon se sentiu frustrado e envergonhado. Ele então teve a ideia de criar um desafio para as pessoas comuns se tornarem "celebridades" de suas próprias causas, o que levou a um aumento significativo no número de doações.

Ele também fala sobre a criação do "Cartão do Bem", um vale-presente que permite às pessoas doar dinheiro para causas sociais. Ele destaca a importância da conectividade e da divulgação da ideia de doar, e como a internet pode ser usada como uma ferramenta para o bem.

No final, Kevin Bacon conclui que a criatividade e a inovação são essenciais para mudar o mundo e que a conectividade é o exemplo mais poderoso da força das pessoas.

In [ ]:
url_video = "https://www.youtube.com/watch?v=XXcHDy3QH-E" # @param {type:"string"}
query_user = "sumarize de forma clara de entender" # @param {type:"string"}
model_class = "hf_hub" # @param ["hf_hub", "openai", "ollama"]
language = ["pt", "pt-BR", "en"] # @param {type:"string"}

interpret_video(url_video, query_user, model_class, language)

## Informações do vídeo

    Título: Como teoria do caos e efeito borboleta ajudam a explicar Universo
    Autor: BBC News Brasil
    Data: 2022-08-21
    URL: https://www.youtube.com/watch?v=XXcHDy3QH-E

    


## Sobre o que fala o vídeo 


O vídeo fala sobre o efeito borboleta, uma manifestação da Teoria do Caos, que demonstra como pequenas mudanças podem ter consequências significativas e imprevisíveis no futuro.


## Temas 


Os principais temas desse vídeo são:

1. Efeito Borboleta (Butterfly Effect): a ideia de que pequenas mudanças podem ter consequências significativas e imprevisíveis.
2. Teoria do Caos: a teoria que questiona a ideia de que o universo é determinista e que pequenas mudanças podem ter um impacto significativo no comportamento de sistemas complexos.
3. Edward Lorenz: o meteorologista que desenvolveu a teoria do caos e demonstrou que pequenas mudanças podem ter um impacto significativo no comportamento de sistemas complexos.
4. Incerteza e Previsibilidade: a discussão sobre a impossibilidade de prever o futuro com precisão, mesmo com o conhecimento das leis da natureza.
5. Aplicação prática da Teoria do Caos: a discussão sobre como a teoria do caos pode ser aplicada em diferentes áreas, como a economia, a medicina e a sociologia.
6. Limites do Conhecimento: a discussão sobre como a teoria do caos coloca em xeque os limites do nosso conhecimento e nos faz questionar a ideia de que podemos prever o futuro com precisão.


## Resposta para a consulta 


Aqui está uma síntese clara e fácil de entender da transcrição:

A teoria do caos é um conceito que questiona a ideia de que o mundo é previsível e determinista, como proposto por Isaac Newton. Em 1961, o meteorologista Edward Lorenz descobriu que pequenas diferenças iniciais em um modelo matemático para prever o tempo poderiam levar a resultados muito diferentes. Isso levou a conclusão de que o caos é uma característica fundamental do universo, onde pequenas mudanças podem ter efeitos significativos e imprevisíveis.

O efeito borboleta é um exemplo disso, onde uma pequena mudança, como o bater de asas de uma borboleta, pode causar uma perturbação que leva a consequências muito maiores, como um furacão. Isso significa que, apesar de conhecemos as leis da natureza, não há medições que nos deem dados perfeitos e nos permitam prever o futuro com certeza.

No entanto, o caos não é o mesmo que falta de ordem. O universo ainda segue as leis naturais, e os padrões dentro do caos têm muitas aplicações práticas, como na análise de fenômenos sociais, no mercado financeiro e no comportamento humano.

A teoria do caos introduz um elemento de incerteza na nossa leitura do universo e coloca em xeque os limites do nosso conhecimento. No entanto, isso não significa que o mundo seja aleatório, mas sim que é complexo e imprevisível.

## Reconhecimento de fala

Se o vídeo não tiver uma transcrição disponível, será necessário gerar uma de forma automática utilizando um modelo de reconhecimento de fala, também conhecido como Speech-to-Text (STT). Esses modelos convertem fala em texto a partir de arquivos de áudio, permitindo que o conteúdo do vídeo seja transcrito para ser processado pela nossa aplicação de sumarização.

Um exemplo popular de modelo de Speech-to-Text é o [Whisper](https://openai.com/index/whisper/), da OpenAI, que oferece uma solução robusta para transcrição automática. Os preços e detalhes sobre o uso desse modelo podem ser encontrados na página de "[pricing](https://openai.com/api/pricing/)" da OpenAI. No entanto, neste projeto, optamos por não utilizá-lo, pois todos os vídeos testados já possuíam transcrições, muitas vezes geradas automaticamente pelo YouTube. Essa abordagem economiza processamento, já que o foco principal foi a implementação de modelos de linguagem grandes (LLMs) e a exploração de ferramentas do Langchain.

* Se você precisar integrar um serviço de transcrição, a implementação é simples, podendo ser feita com uma chamada de função. A documentação do Langchain oferece instruções detalhadas sobre como realizar essa integração aqui: https://js.langchain.com/v0.2/docs/integrations/document_loaders/file_loaders/openai_whisper_audio/.

* Na verdade você tem a liberdade de escolher qualquer serviço ou código para realizar o reconhecimento de fala. Mesmo que o serviço não se integre diretamente ao Langchain, isso não é um problema, pois o essencial é obter o texto final. No final, o que você obtém é um texto normal, que pode ser armazenado em um arquivo de texto ou diretamente em uma variável.

* Entretanto, se desejar aproveitar soluções prontas dentro do Langchain, você pode utilizar o Whisper da OpenAI ou outras opções como o [GoogleSpeechToText](https://python.langchain.com/v0.2/docs/integrations/document_loaders/google_speech_to_text/) e o [AssemblyAI Audio Transcripts](https://python.langchain.com/v0.2/docs/integrations/document_loaders/assemblyai/), que também são altamente eficazes e fáceis de integrar ao langchain.

## Indo além 🚀

1. Conforme mencionado anteriormente, para vídeos muito longos ou transcrições maiores, pode ser necessário dividir a transcrição em partes menores e aplicar técnicas de indexação e recuperação utilizando RAG.
 * Isso é essencial caso o vídeo que deseja processar possua horas por exemplo - talvez modelos muito modernos e com janela de contexto maior consigam lidar, mas mesmo que consigam pode ser interessante aplicar essas técnicas, principalmente caso o que busque não seja a sumarização mas sim algo próximo do nosso projeto de perguntas e respostas com documentos, que será visto no projeto 3.
 * Caso deseje seguir esse caminho, basta reutilizar o código já pronto no projeto 3, que lida com esse tipo de segmentação e recuperação de forma eficiente. Assim, você poderá manter a integridade da transcrição, otimizando o processamento e a geração de respostas mais precisas, mesmo com grandes volumes de dados.

2. Como ideia adicional ou desafio: é possível personalizar este projeto para atender a outras necessidades, ou ainda usar a sumarização como uma etapa posterior dentro de uma pipeline maior.
  * Um exemplo seria o uso da ferramenta [YouTubeSearchTool](https://python.langchain.com/v0.2/docs/integrations/tools/youtube//) do Langchain, que permite buscar automaticamente uma lista de URLs de vídeos no YouTube com base em um tema ou palavra-chave fornecida pelo usuário.
  * Nessa abordagem, você poderia implementar uma aplicação que solicita um termo de busca e, em seguida, utiliza essa ferramenta para buscar vídeos relevantes, retornando em uma lista. Por fim, basta criar um laço de repetição que execute a função interpret_video (que criamos nesse projeto) para cada vídeo da lista, realizando assim a sumarização de múltiplos vídeos associados a um tema de forma automatizada.

3. Além disso, é possível integrar essa funcionalidade a uma interface interativa com ferramentas como Streamlit, que permite criar facilmente interfaces gráficas.
 * Os métodos para essa integração estão descritos detalhadamente nos projetos 2 e 3, caso queira expandir a aplicação.
 * Embora, neste exemplo, tenhamos optado por usar o Google Colab pela conveniência e para demonstrar a exibição com Markdown, a migração para uma interface mais robusta pode ser feita com facilidade e oferecer uma experiência mais fluida para o usuário final caso deseje publicar essa aplicação.
